## Add utils as a module and import

In [1]:
import sys

try:
    from utils import data_loading
except:
    sys.path.append('../utils')
    sys.path.append('../')
    from utils import data_loading

# Loading libraries

In [2]:
# import os
import numpy as np
from utils import model
# from pathlib import Path
# from PIL import Image'
# from tqdm import tqdm
# import matplotlib
# from matplotlib import pyplot as plt
# from nilearn import datasets
# from nilearn import plotting
import torch
# from torch.utils.data import DataLoader, Dataset
# from torchvision.models.feature_extraction import create_feature_extractor, get_graph_node_names
from torchvision import transforms
# from sklearn.decomposition import IncrementalPCA
# from sklearn.linear_model import LinearRegression
# from scipy.stats import pearsonr as corr
from sklearn.decomposition import PCA

# Loading Data

In [3]:
#loading data for subject 1, first 10 images
lh, rh, images  = data_loading.load_subject_data(1, 0, 200)

print(lh.shape)
print(rh.shape)
print(images[0].shape)

(200, 19004)
(200, 20544)
(425, 425, 3)


In [4]:
#concat lh and rh
brain = np.concatenate((lh, rh), axis=1)
print(brain.shape)

(200, 39548)


In [5]:
dataset = data_loading.CustomDataset(images_list= images, outputs_list= brain, transform=transforms.ToTensor(), PCA = PCA(n_components = 100))

In [6]:
#Create a train and validation subset of variable dataset with torch
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])
#put train dataset into a loader with 2 batches and put test data in val loader
train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=16, shuffle=True)
val_loader = torch.utils.data.DataLoader(test_dataset, shuffle=True)

In [7]:
reg_model = model.get_pretrained_regression_model(100)
trainer = model.Trainer()
optimizer = torch.optim.Adam
loss = torch.nn.MSELoss()
trainer.compile(reg_model, optimizer, learning_rate=0.0001, loss_fn=loss)

In [8]:
trainer.fit(num_epochs= 2, train_loader=train_loader, val_loader=val_loader)

Epoch [1/2], Training Loss: 170.70301513671876
Validation Loss: 183.5020439147949
Epoch [2/2], Training Loss: 167.72671508789062
Validation Loss: 183.56376399993897


In [9]:
trainer.history

{'train_loss': [170.70301513671876, 167.72671508789062],
 'val_loss': [183.5020439147949, 183.56376399993897]}

In [14]:
trainer.save('initial_model.pt')

Model and training history saved to initial_model.pt


In [16]:
del trainer

In [27]:
trainer  = model.Trainer()
trainer.compile(reg_model, optimizer, learning_rate=0.0001, loss_fn=loss)

In [28]:
trainer.load('initial_model.pt')

Model and training history loaded from initial_model.pt


In [29]:
trainer.history

{'train_loss': [170.70301513671876, 167.72671508789062],
 'val_loss': [183.5020439147949, 183.56376399993897]}